In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data 
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image, ImageFile


## DataLoader

In [33]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

def check_image(path):
    """Check if the image is valid or not"""
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [34]:
img_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [35]:
train_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_path, transform=img_transforms, is_valid_file=check_image)

val_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_path, transform=img_transforms, is_valid_file=check_image)

test_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_path, transform=img_transforms, is_valid_file=check_image)

In [36]:
batch_size = 64

In [37]:
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

## Model

In [38]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.fc1 = nn.Linear(64*64*3,84)
        self.fc2 = nn.Linear(84,50)
        self.fc3 = nn.Linear(50,2)
    def forward(self,x):
        x = x.view(-1,64*64*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [39]:
SimpleNet = SimpleNet()

In [40]:
optimizer = optim.Adam(SimpleNet.parameters(), lr = 0.003)

In [41]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
SimpleNet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

## Training

In [42]:
def train(model, optimizer, criterion, train_loader, val_loader, num_epochs, device = "cpu"):
    for epoch in range(num_epochs):
        model.train()
        trainning_loss = 0.0
        valid_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            trainning_loss += loss.item() * inputs.size(0)
        trainning_loss = trainning_loss / len(train_loader.dataset)
        
        num_correct = 0
        num_examples = 0
        model.eval()
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = criterion(output, targets)
            valid_loss += loss.item() * inputs.size(0)
            correct  =torch.eq(torch.max(F.softmax(output, dim=1),1)[1],targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        accuracy = num_correct / num_examples
        valid_loss = valid_loss / len(val_loader.dataset)
        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, trainning_loss, valid_loss, accuracy))           

In [43]:
train(SimpleNet, optimizer, nn.CrossEntropyLoss(), train_loader, val_loader, 2, device)

Epoch: 0, Training Loss: 1.14, Validation Loss: 0.93, accuracy = 0.65
Epoch: 1, Training Loss: 0.63, Validation Loss: 1.76, accuracy = 0.68


## Making Predictions

In [44]:
label = ['cat', 'fish']

img = Image.open('./test/cat/3156111_a9dba42579.jpg')
img = img_transforms(img).to(device)
img = img.unsqueeze(0)

SimpleNet.eval()
prediction = F.softmax(SimpleNet(img), dim=1)
prediction = prediction.argmax()

print(label[prediction.item()])

cat


In [45]:
num_correct = 0
num_examples = 0
for batch in test_loader:
    inputs, targets = batch
    inputs = inputs.to(device)
    targets = targets.to(device)
    output = SimpleNet(inputs)
    correct = torch.eq(torch.max(F.softmax(output, dim=1),1)[1],targets)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]

print('Accuracy on test data: {:.2f}'.format(num_correct / num_examples))

Accuracy on test data: 0.68
